__собираем данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

2816


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

2816


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
464,2208336892,"дом 95 м² на участке 4,5 сот.",5300000,,"Севастополь, улица Ревякина, 88","Дом 95 м2 на участке 4,5 сотки.\n\nДом 95 м2 на участке 4,5 сотки, двухэтажный. Расположен в г. Севастополь, Нахимовский район, улица Ревякина. Коммуникации: 5 квт электричество, водоснабжение городское, отопление газовое, газоснабжение, канализация городска",10,2022-09-15 11:53:29.976,sevastopol,95.00,False,False,False,4.50,5.30,4-8,70-150
128,2496530431,дом 28 м² на участке 6 сот.,4300000,,"садоводческое товарищество Кальфа-1, 339","Арт. 30252029 В Снт Кальфа -1 продаётся жилой дом площадью 28квм на 6 сотках земли. Дом построен из инкерманского камня, утеплен, железобетонные перекрытия, добротный фундамент, подвал. Свет и кооперативная вода заведены, металлическая емкость 8 куб. М Уча",3,2022-09-15 11:53:29.976,sevastopol,28.00,False,False,True,6.00,4.30,4-8,<30
2297,2518404616,дом 150 м² на участке 4 сот.,10000000,,"садоводческое товарищество Медик-88, 320","Хороший капитальный жилой дом со всеми коммуникациями. Построен из ракушки, утеплён пенопластом, сделан фасад. Под всем домом полуподвал во весь рост. Есть своя глубинная скважина. Отопление газовый котёл, батареи по всему дому, газовые баллоны. Так же ест",48,2022-09-15 11:53:29.976,sevastopol,150.00,False,False,True,4.00,10.00,2-4,70-150


In [11]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             2816 non-null   int64         
 1   title                2816 non-null   object        
 2   price                2816 non-null   int64         
 3   obj_name             2816 non-null   object        
 4   adr                  2816 non-null   object        
 5   description          2816 non-null   object        
 6   avito_page           2816 non-null   int64         
 7   ts                   2816 non-null   datetime64[ns]
 8   place                2816 non-null   object        
 9   house_area           2816 non-null   float64       
 10  is_part              2816 non-null   bool          
 11  is_townhouse         2816 non-null   bool          
 12  is_SNT               2816 non-null   bool          
 13  land_area            2816 non-nul

In [14]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                    False
description            False
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [15]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [16]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

2816


## обновляем таблицу адресов

In [17]:
# data = data.sample(10)

In [18]:
# import pandas as pd
# loc_file_path = 'data/location_yandex.pkl'# таблица адресов
# df=pd.read_pickle(loc_file_path)
# # df

# df['latitude_'] =  df['latitude']
# df['latitude']  =  df['longitude']
# df['longitude'] = df['latitude_']
# df = df.drop(columns=['latitude_',])

# df.to_pickle(loc_file_path)

In [19]:
# from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev

# loc_file_path = 'data/location.pkl'# таблица адресов

# loc = LocationUpdater(
#         address_transformer=AddressTransformerSev(),
#     ).transform(
#         adr=data.sample(3)['adr'],
#         loc=pd.read_pickle(loc_file_path.pkl'),
#         show_pbar=True,
#     )

----

In [20]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = {
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v)
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

bahchisaray Бахчисарай,
sudak Судак,
sevastopol Севастополь,


----

In [21]:
from lib.locator import LocationUpdater
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-15 23:11:42 | LocationUpdater: 9821 addresses in location table
[INFO    ] 2022-09-15 23:11:42 | LocationUpdater: 9821 addresses total
[INFO    ] 2022-09-15 23:11:42 | LocationUpdater: 9821 addresses defined
[INFO    ] 2022-09-15 23:11:42 | LocationUpdater: 0 addresses undefined


In [22]:
loc

,adr,latitude,longitude,truncated
0,"Севастополь, улица Токарева, 7 этап, блок-секция 3",44.78,33.55,False
1,"Севастополь, улица Токарева, 7 этап, блок-секция 1",44.51,33.60,False
2,"Севастополь, улица Генерала Жидилова, 4",44.61,33.52,False
3,"Севастополь, улица Николая Музыки, 96",44.85,33.63,False
4,"Севастополь, улица Лётчиков, д. 10",44.58,33.52,False
...,...,...,...,...
9816,"Судак, квартал Асрет",44.87,34.96,False
9817,"Судак, ул. Ашик-Умер",44.85,35.00,False
9818,"Судак, с. Солнечная Долина, ул. Авдет",44.86,35.12,False
9819,"Севастополь, Севастопольская зона ЮБК, 27",44.42,33.69,False


In [23]:
loc.sample(3)

,adr,latitude,longitude,truncated
8955,"Севастополь, садоводческое товарищество Чайка, 236",44.74,33.55,False
350,"Севастополь, Камышовое шоссе , 37Бк2",44.60,33.56,False
9664,"Бахчисарай, ул. Ашика Умера, 2",44.74,33.87,False


In [24]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [25]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 2821
записей без геометки: 0


In [26]:
assert len(data)>0
data.to_pickle(result_file_path)

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2821 entries, 0 to 2820
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             2821 non-null   int64         
 1   title                2821 non-null   object        
 2   price                2821 non-null   int64         
 3   obj_name             2821 non-null   object        
 4   adr                  2821 non-null   object        
 5   description          2821 non-null   object        
 6   avito_page           2821 non-null   int64         
 7   ts                   2821 non-null   datetime64[ns]
 8   place                2821 non-null   object        
 9   house_area           2821 non-null   float64       
 10  is_part              2821 non-null   bool          
 11  is_townhouse         2821 non-null   bool          
 12  is_SNT               2821 non-null   bool          
 13  land_area            2821 non-nul

In [ ]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)

In [ ]:
# class AvitoDataCleanerRealtyLand:
    
#     def transform(self,data):
#         df = data.copy()
        
#         symb_lat = 'yexapocEXAPOCTHKBM'
#         symb_rus = 'уехаросЕХАРОСТНКВМ'
#         l2r = str.maketrans(symb_lat,symb_rus) 
        
#         df['title'] = df['title'].str.extract( r'«(.*)»',expand=False).apply(lambda s: s.translate(l2r) )
        
#         area = df['title'].str.extract( r'(\d+,?\d*)\s*(сот|га)',expand=True)
#         area.columns=['area','area_unit']
#         df = pd.concat([df,area],axis=1)
                       
#         df['is_IJS'] = df['title'].str.lower().str.match(r'.*ижс.*')
#         df['obj_name'] = df['obj_name'].fillna('')
#         df['price'] = df['price'].astype(int)
#         df['priceM'] = df['price']/1e6
        
#         df['area'] = df['area'].fillna('0.').str.replace(',','.').astype(float)
        
#         df.loc[ df['area_unit']=='га', 'area' ] = df.query('area_unit=="га"')['area']*100.
#         df = df.drop(columns=['area_unit'])
        
#         return df

In [ ]:
# data_avito['title'].str.extract( r'(\d+,?\d*)\s*(сот|га)',expand=True) #, columns=['a','b'])
        

In [ ]:
# # import re
# # s='Участок 9,7 сот. (ИЖС)'
# s='Участок 7,8 сот. (ИЖС)'
# s='Участок 7,8 га. (ИЖС)'

# s='Участок 7,8 м. (ИЖС)'

# # s='Участок 5 сот. (ИЖС)'

# re.search( r'(\d+,?\d*)\s*(сот|га)',s) #.group(1)

# # # len(s)

----

In [ ]:
## собираем данные циан.ру

In [ ]:
# import re
# from os import listdir

# raw_data_files = sorted([
#         cian_raw_data_path+'/'+f 
#         for f in listdir(cian_raw_data_path) 
#         if re.match(r'.+\.pkl$',f)
#         #if re.match(r'.+\.xlsx$',f)
    
#     ])
# raw_data_files

In [ ]:
# data_cian = pd.concat([
#         # pd.read_excel(f)
#         pd.read_pickle(f)
#          .dropna()
#          .drop_duplicates(['LinkArea','Description',])
#          .rename(columns={'page':'cian_page'})
#          .reset_index(drop=True)  
#         for f in raw_data_files
#     ]).reset_index(drop=True)

In [ ]:
# data_cian.info(2)

In [ ]:
# swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

# data_cian = pd.concat([
#         data_cian[ ~swap_title ],
#         data_cian[ swap_title ]
#             .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
#     ]).reset_index(drop=True)


In [ ]:
# data_cian

In [ ]:
# from lib.cian import CianDataCleaner

# data_cian = CianDataCleaner().transform( data_cian )

# print(len(data_cian))

In [ ]:
# assert len(data_cian)>0

In [ ]:
# data_cian.info(2)

In [ ]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [ ]:
# data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
# print(len(data))
# assert len(data)>0